<a href="https://colab.research.google.com/github/NitishHatty/Gl_Capstone_Project/blob/main/Model_Building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#CSV Details:

#it has just product as "bank_account_or_service"

In [ ]:
##for better understanding, we have considered just 'Bank account or service', as of now

In [113]:
#predict the issue based on consumer complaint narrative

In [114]:
#where issue related to 'Deposits and withdrawals' are considered to be 1 and rest ( 'Account opening, closing, or management','Using a debit or ATM card',
#'Problems caused by my funds being low',
#'Making/receiving payments, sending money') as 0

In [116]:
#Building a Model to predict the "issue"- either 1 or 0 based on "Consumer Complaint Narrative"

In [117]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import string
import nltk
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import GridSearchCV

In [65]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [67]:
stopword = nltk.corpus.stopwords.words('english')

In [24]:
#Functions used :
#Label Encoding
#Punctuation Removal
#Number Removal
#Tokenizing
#Lower Case Conversion
#Stop words
#Token Concatenation

In [25]:
#Label Encoding 

In [42]:
def label_encoding(i):
    if i == 'Deposits and withdrawals':
        return 1
    else :
        return 0

In [27]:
#Punctuation Removal

In [29]:
def remove_punctuation(text):
    words = [word for word in text if word not in string.punctuation]
    text = ''.join(words)
    return text

In [40]:
#Number Removal

In [30]:
def remove_numbers(text):
    words = [word for word in text if not word.isdigit()]
    text = ''.join(words)
    return text

In [32]:
#Lower Case Conversion

In [33]:
def convert_to_lower(tokens):
    tokens=[word.lower() for word in tokens]
    return tokens

In [35]:
#Stopwords

In [36]:
def remove_stopwords(tokens):
    tokens=[word for word in tokens if word not in stopword]
    return tokens


In [37]:
#Token Concatenation

In [38]:
def concate_tokens(token):
    sentance = ''
    for i in token:
        sentance = sentance+i+' '
    return sentance    

In [41]:
CV = CountVectorizer()

In [6]:
df = pd.read_csv("bank_account_or_service.csv",index_col=0,error_bad_lines=False)

In [7]:
df.shape

(5711, 18)

In [9]:
df.columns

Index(['date_received', 'product', 'sub_product', 'issue', 'sub_issue',
       'consumer_complaint_narrative', 'company_public_response', 'company',
       'state', 'zipcode', 'tags', 'consumer_consent_provided',
       'submitted_via', 'date_sent_to_company', 'company_response_to_consumer',
       'timely_response', 'consumer_disputed?', 'complaint_id'],
      dtype='object')

In [10]:
pd.set_option("display.max_colwidth", None)

In [12]:
df.issue.unique()

array(['Deposits and withdrawals',
       'Account opening, closing, or management',
       'Using a debit or ATM card',
       'Problems caused by my funds being low',
       'Making/receiving payments, sending money'], dtype=object)

In [20]:
df_new1 = df[["issue","consumer_complaint_narrative"]]

In [21]:
df_new1['consumer_complaint_narrative'] =  df.consumer_complaint_narrative.apply(lambda i : str(i) )

In [22]:
df_new1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5711 entries, 192044 to 553037
Data columns (total 2 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   issue                         5711 non-null   object
 1   consumer_complaint_narrative  5711 non-null   object
dtypes: object(2)
memory usage: 133.9+ KB


In [50]:
df_new1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5711 entries, 192044 to 553037
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   issue                         5711 non-null   object
 1   consumer_complaint_narrative  5711 non-null   object
 2   target                        5711 non-null   int64 
 3   narrative_wo_punct            5711 non-null   object
dtypes: int64(1), object(3)
memory usage: 223.1+ KB


In [53]:
df_new1['target'] = df_new1['issue'].map(label_encoding)

In [54]:
df_new1.target.value_counts()

0    4310
1    1401
Name: target, dtype: int64

In [55]:
df_new1[df_new1.issue == 'Deposits and withdrawals'].shape

(1401, 4)

In [56]:
df_new1['narrative_wo_punct'] = df_new1['consumer_complaint_narrative'].map(remove_punctuation)

In [57]:
df_new1['narrative_wo_punct'].head(1)

192044    I opened an account with M  T bank online on XXXXXXXX15 and immediately funded it with an automated transfer of 50000 A little over a week later I got a letter requesting that I fax in a copy of my DL Social and utility bill for address verification which I did on XXXXXXXX15 Fast forward to today XXXXXXXX15 and I have not heard anything back so I called M  T bank to find out the status and I was told that they still have not reviewed it and my 50000 was on hold I demanded that my account get shut down and my 50000 returned but I was told that could not happen until they reviewed my information that was faxed and released the hold on my account I asked how long that would take and I was told it would take as long as they want which would render my 50000 hostage until they got around to figuring stuff out \n
Name: narrative_wo_punct, dtype: object

In [58]:
df_new1['narrative_wo_numbers'] = df_new1['narrative_wo_punct'].map(remove_numbers)

In [59]:
df_new1['narrative_wo_numbers'].head(1)

192044    I opened an account with M  T bank online on XXXXXXXX and immediately funded it with an automated transfer of  A little over a week later I got a letter requesting that I fax in a copy of my DL Social and utility bill for address verification which I did on XXXXXXXX Fast forward to today XXXXXXXX and I have not heard anything back so I called M  T bank to find out the status and I was told that they still have not reviewed it and my  was on hold I demanded that my account get shut down and my  returned but I was told that could not happen until they reviewed my information that was faxed and released the hold on my account I asked how long that would take and I was told it would take as long as they want which would render my  hostage until they got around to figuring stuff out \n
Name: narrative_wo_numbers, dtype: object

In [60]:
df_new1['tokens'] = df_new1['narrative_wo_numbers'].map(word_tokenize)


In [61]:
df_new1['tokens'].head(1)

192044    [I, opened, an, account, with, M, T, bank, online, on, XXXXXXXX, and, immediately, funded, it, with, an, automated, transfer, of, A, little, over, a, week, later, I, got, a, letter, requesting, that, I, fax, in, a, copy, of, my, DL, Social, and, utility, bill, for, address, verification, which, I, did, on, XXXXXXXX, Fast, forward, to, today, XXXXXXXX, and, I, have, not, heard, anything, back, so, I, called, M, T, bank, to, find, out, the, status, and, I, was, told, that, they, still, have, not, reviewed, it, and, my, was, on, hold, I, demanded, that, my, account, get, shut, down, and, ...]
Name: tokens, dtype: object

In [62]:
df_new1['tokens_to_lower'] = df_new1['tokens'].map(convert_to_lower)

In [63]:
df_new1['tokens_to_lower'].head(1)

192044    [i, opened, an, account, with, m, t, bank, online, on, xxxxxxxx, and, immediately, funded, it, with, an, automated, transfer, of, a, little, over, a, week, later, i, got, a, letter, requesting, that, i, fax, in, a, copy, of, my, dl, social, and, utility, bill, for, address, verification, which, i, did, on, xxxxxxxx, fast, forward, to, today, xxxxxxxx, and, i, have, not, heard, anything, back, so, i, called, m, t, bank, to, find, out, the, status, and, i, was, told, that, they, still, have, not, reviewed, it, and, my, was, on, hold, i, demanded, that, my, account, get, shut, down, and, ...]
Name: tokens_to_lower, dtype: object

In [68]:
df_new1['wo_stopwords'] = df_new1['tokens_to_lower'].map(remove_stopwords)

In [71]:
df_new1.wo_stopwords.head(1)

192044    [opened, account, bank, online, xxxxxxxx, immediately, funded, automated, transfer, little, week, later, got, letter, requesting, fax, copy, dl, social, utility, bill, address, verification, xxxxxxxx, fast, forward, today, xxxxxxxx, heard, anything, back, called, bank, find, status, told, still, reviewed, hold, demanded, account, get, shut, returned, told, could, happen, reviewed, information, faxed, released, hold, account, asked, long, would, take, told, would, take, long, want, would, render, hostage, got, around, figuring, stuff]
Name: wo_stopwords, dtype: object

In [72]:
df_new1['final_text'] = df_new1['wo_stopwords'].map(concate_tokens)

In [73]:
df_new1.final_text.head(1)


192044    opened account bank online xxxxxxxx immediately funded automated transfer little week later got letter requesting fax copy dl social utility bill address verification xxxxxxxx fast forward today xxxxxxxx heard anything back called bank find status told still reviewed hold demanded account get shut returned told could happen reviewed information faxed released hold account asked long would take told would take long want would render hostage got around figuring stuff 
Name: final_text, dtype: object

In [75]:
#Independent and Target Variables

In [80]:
features_var = df_new1.final_text
target_var = df_new1.target

In [81]:
xtrain, xtest, ytrain, ytest = train_test_split(features_var,target_var, test_size=.2,random_state=100)

In [82]:
xtrain.shape, xtest.shape, ytrain.shape, ytest.shape

((4568,), (1143,), (4568,), (1143,))

In [79]:
toNumeric = CountVectorizer()

In [83]:
#Get the vocabulary(unique words) from training data
toNumeric.fit(xtrain)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [84]:
# Test the created vocabulary
len(toNumeric.get_feature_names())

15652

In [85]:
# Convert training data into a 'document-term matrix'
sampleTrain_dtm = toNumeric.transform(xtrain)
sampleTrain_dtm

<4568x15652 sparse matrix of type '<class 'numpy.int64'>'
	with 313194 stored elements in Compressed Sparse Row format>

In [87]:
# Let's convert sparse matrix to a dense matrix
sampleTrain_dtm.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [88]:
pd.DataFrame(sampleTrain_dtm.toarray(), columns=toNumeric.get_feature_names()).head(1)

,aaccount,aacount,aba,aback,aballished,abandon,abandoned,abank,abarouting,abatement,abc,abd,abetted,abetting,abide,abided,abiding,abilities,ability,abillionaire,able,ableto,abm,abnormality,abnormally,abominably,abomination,abort,abortive,abother,aboutwhere,abovementioned,abovethen,abrasive,abrasiveness,abreast,abroad,abrubtly,abrupt,abruptly,...,yhat,yield,yielded,yiu,yo,york,yorki,yorksince,youamex,young,younger,youraccount,yourbusiness,yourcomputer,youre,yourpayment,youshall,yousincerelyxxxx,youth,youto,youtoday,youve,youxxxx,youxxxxxxxx,yr,yrs,yuck,yur,yused,yyour,zed,zero,zeroed,zeros,zi,zion,zions,zip,zippered,zone
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [145]:
model1 = KNeighborsClassifier(n_neighbors=3)
model1.fit(sampleTrain_dtm, ytrain)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [146]:
# Transform test data into DTM by using generated vocabulary
sampleTest_dtm = toNumeric.transform(xtest)
len(sampleTest_dtm.toarray())

1143

In [147]:
# Test built model
y_pred_test = model1.predict(sampleTest_dtm)

In [148]:
y_pred_test.shape

(1143,)

In [149]:
ytest.shape

(1143,)

In [150]:
print(classification_report(ytest, y_pred_test))

              precision    recall  f1-score   support

           0       0.79      0.85      0.82       866
           1       0.39      0.30      0.34       277

    accuracy                           0.72      1143
   macro avg       0.59      0.58      0.58      1143
weighted avg       0.70      0.72      0.70      1143



In [151]:
df_output = pd.DataFrame({
    'y actual' : ytest,
    ' y predicted' : y_pred_test
})

In [152]:
df_output.head(5)

,y actual,y predicted
228491,0,0
317301,0,0
533005,0,1
308706,0,1
324088,0,0


In [153]:
for i in df_output.columns:
  print(i,df_output[i].value_counts(),sep="\n")

y actual
0    866
1    277
Name: y actual, dtype: int64
 y predicted
0    935
1    208
Name:  y predicted, dtype: int64


In [120]:
tuned_paramaters = {'n_neighbors': np.arange(1, 25, 2),
                   'metric': ['hamming','euclidean','manhattan','Chebyshev'],
                    "weights":["uniform", "distance"]}
knn_classification = KNeighborsClassifier()
knn_grid = GridSearchCV(estimator = knn_classification, 
                        param_grid = tuned_paramaters, 
                        cv = 5, 
                        scoring = 'accuracy',verbose=2)

# fit the model on X_train and y_train using fit()
knn_grid.fit(sampleTrain_dtm, ytrain)

# get the best parameters
print('Best parameters for KNN Classifier: ', knn_grid.best_params_, '\n')

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV] metric=hamming, n_neighbors=1, weights=uniform ..................
[CV] ... metric=hamming, n_neighbors=1, weights=uniform, total=   0.0s
[CV] metric=hamming, n_neighbors=1, weights=uniform ..................
[CV] ... metric=hamming, n_neighbors=1, weights=uniform, total=   0.0s
[CV] metric=hamming, n_neighbors=1, weights=uniform ..................
[CV] ... metric=hamming, n_neighbors=1, weights=uniform, total=   0.0s
[CV] metric=hamming, n_neighbors=1, weights=uniform ..................
[CV] ... metric=hamming, n_neighbors=1, weights=uniform, total=   0.0s
[CV] metric=hamming, n_neighbors=1, weights=uniform ..................
[CV] ... metric=hamming, n_neighbors=1, weights=uniform, total=   0.0s
[CV] metric=hamming, n_neighbors=1, weights=distance .................
[CV] .. metric=hamming, n_neighbors=1, weights=distance, total=   0.0s
[CV] metric=hamming, n_neighbors=1, weights=distance .................
[CV] .. metric=

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] ... metric=hamming, n_neighbors=9, weights=uniform, total=   0.0s
[CV] metric=hamming, n_neighbors=9, weights=uniform ..................
[CV] ... metric=hamming, n_neighbors=9, weights=uniform, total=   0.0s
[CV] metric=hamming, n_neighbors=9, weights=uniform ..................
[CV] ... metric=hamming, n_neighbors=9, weights=uniform, total=   0.0s
[CV] metric=hamming, n_neighbors=9, weights=uniform ..................
[CV] ... metric=hamming, n_neighbors=9, weights=uniform, total=   0.0s
[CV] metric=hamming, n_neighbors=9, weights=distance .................
[CV] .. metric=hamming, n_neighbors=9, weights=distance, total=   0.0s
[CV] metric=hamming, n_neighbors=9, weights=distance .................
[CV] .. metric=hamming, n_neighbors=9, weights=distance, total=   0.0s
[CV] metric=hamming, n_neighbors=9, weights=distance .................
[CV] .. metric=hamming, n_neighbors=9, weights=distance, total=   0.0s
[CV] metric=hamming, n_neighbors=9, weights=distance .................
[CV] .

[Parallel(n_jobs=1)]: Done 480 out of 480 | elapsed:  5.7min finished


In [121]:
#Best parameters for KNN Classifier:  {'metric': 'euclidean', 'n_neighbors': 17, 'weights': 'uniform'} 

In [132]:
#by default weight is set to uniform and metric as euclidean as p value is 2 
model2 = KNeighborsClassifier(n_neighbors=17)
model2.fit(sampleTrain_dtm, ytrain)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=17, p=2,
                     weights='uniform')

In [133]:
# Transform test data into DTM by using generated vocabulary
sampleTest_dtm = toNumeric.transform(xtest)
len(sampleTest_dtm.toarray())

1143

In [138]:
# Test built model
y_pred_test_2 = model2.predict(sampleTest_dtm)

In [139]:
y_pred_test_2.shape

(1143,)

In [140]:
ytest.shape

(1143,)

In [166]:
metrics.accuracy_score(ytest, y_pred_test_2)

0.7664041994750657

In [141]:
print(classification_report(ytest, y_pred_test_2))

              precision    recall  f1-score   support

           0       0.78      0.97      0.86       866
           1       0.58      0.14      0.22       277

    accuracy                           0.77      1143
   macro avg       0.68      0.55      0.54      1143
weighted avg       0.73      0.77      0.71      1143



In [154]:
df_output = pd.DataFrame({
    'y actual' : ytest,
    ' y predicted 1' : y_pred_test,
    ' y predicted 2' : y_pred_test_2
})

In [155]:
df_output.tail(5)

,y actual,y predicted 1,y predicted 2
543992,0,1,0
221036,0,0,0
316889,0,0,0
231891,0,0,0
505925,0,0,0


In [157]:
for i in df_output.columns:
  print(i,df_output[i].value_counts(1),sep="\n")

y actual
0    0.757655
1    0.242345
Name: y actual, dtype: float64
 y predicted 1
0    0.818023
1    0.181977
Name:  y predicted 1, dtype: float64
 y predicted 2
0    0.942257
1    0.057743
Name:  y predicted 2, dtype: float64


In [158]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(sampleTrain_dtm, ytrain)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [162]:
y_pred_class_1 = nb.predict(sampleTest_dtm)

In [163]:
# Accuracy calculation
from sklearn import metrics
metrics.accuracy_score(ytest, y_pred_class_1)

0.7760279965004374

In [165]:
print(classification_report(ytest, y_pred_class_1))

              precision    recall  f1-score   support

           0       0.84      0.88      0.86       866
           1       0.54      0.46      0.50       277

    accuracy                           0.78      1143
   macro avg       0.69      0.67      0.68      1143
weighted avg       0.77      0.78      0.77      1143

